In [ ]:
!pip install pytorch-ignite==0.2.* tensorboardX==1.6.*
import os
import sys
import numpy as np
import random
import torch
import ignite
import os, os.path
import shutil
import cv2
import pandas as pd
import time
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
from sklearn.metrics import accuracy_score
from torch.nn import functional as F
from collections import OrderedDict


seed = 17
random.seed(seed)
_ = torch.manual_seed(seed)

torch.set_printoptions(precision=8,sci_mode=False)
np.set_printoptions(threshold=sys.maxsize)

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

from torchvision import transforms
tfms = transforms.Compose([transforms.Resize(224),
                           transforms.ToTensor(),
                           transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),])

def create_dataset(img_folder):
   
    img_data_array = []
    class_name = []
    itr = 0
    for dir1 in os.listdir(img_folder):
        for file in os.listdir(os.path.join(img_folder, dir1)):
            image_path= os.path.join(img_folder, dir1,  file)
            image= cv2.imread(image_path)
            image=cv2.resize(image, (224, 224),interpolation = cv2.INTER_AREA)
            # image = tfms(image).unsqueeze(0) 
            img_data_array.append(image)
            class_name.append(dir1)
    return img_data_array, class_name

!cp  /content/drive/MyDrive/Kaggle/COVID-19_Radiography_Dataset/TestingData.zip /content
!unzip '/content/TestingData.zip'
!cp  /content/drive/MyDrive/Kaggle/COVID-19_Radiography_Dataset/TrainingData.zip /content
!unzip '/content/TrainingData.zip'

!mv /content/content/drive/MyDrive/Kaggle/COVID-19_Radiography_Dataset/TrainingData /content
!rm -r /content/content
!rm -r '/content/TrainingData.zip'
!rm -r '/content/TestingData.zip'

DIR = '/content/TrainingData/1'
print(len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))]))
DIR = '/content/TrainingData/0'
print(len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))]))

DIR = '/content/TestingData/1'
print(len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))]))
DIR = '/content/TestingData/0'
print(len([name for name in os.listdir(DIR) if os.path.isfile(os.path.join(DIR, name))]))

In [14]:
train_path = '/content/TrainingData'


img_data, class_name = create_dataset (train_path)

target_dict={k: v for v, k in enumerate(np.unique(class_name))}
target_val=  [target_dict[class_name[i]] for i in range(len(class_name))]
class_name = np.array(list(map(int,target_val))).astype('float')

target_val = 0
target_dict = 0
print(len(img_data))

imgs = [torch.from_numpy(item).float().permute(2, 0, 1) for item in img_data]
imgs = torch.stack(imgs)
train_loader = 0
train_sample_ds = torch.utils.data.TensorDataset(imgs, torch.Tensor(class_name.tolist()))
train_loader = torch.utils.data.DataLoader(train_sample_ds, shuffle=True, batch_size=10)

class_name, imgs, train_sample_ds = 0, 0, 0

6000


In [3]:
test_path = '/content/TestingData'


img_data, class_name = create_dataset (test_path)

target_dict={k: v for v, k in enumerate(np.unique(class_name))}
target_val=  [target_dict[class_name[i]] for i in range(len(class_name))]
class_name = np.array(list(map(int,target_val))).astype('float')

target_val = 0
target_dict = 0
print(len(img_data))

imgs = [torch.from_numpy(item).float().permute(2, 0, 1) for item in img_data]
imgs = torch.stack(imgs)

train_sample_ds = torch.utils.data.TensorDataset(imgs, torch.Tensor(class_name.tolist()))
test_loader = torch.utils.data.DataLoader(train_sample_ds, shuffle=True, batch_size=10)

class_name, imgs, train_sample_ds = 0, 0, 0

1232


In [6]:
import torch
import torch.nn as nn

class Swish(nn.Module):
    def forward(self, x):
        return x * torch.sigmoid(x)


class Flatten(nn.Module):
    def forward(self, x):
        return x.reshape(x.shape[0], -1)

class SqueezeExcitation(nn.Module):
    
    def __init__(self, inplanes, se_planes):
        super(SqueezeExcitation, self).__init__()
        self.reduce_expand = nn.Sequential(
            nn.Conv2d(inplanes, se_planes, 
                      kernel_size=1, stride=1, padding=0, bias=True),
            Swish(),
            nn.Conv2d(se_planes, inplanes, 
                      kernel_size=1, stride=1, padding=0, bias=True),
            nn.Sigmoid()
        )

    def forward(self, x):
        x_se = torch.mean(x, dim=(-2, -1), keepdim=True)
        x_se = self.reduce_expand(x_se)
        return x_se * x


class MBConv(nn.Module):
    def __init__(self, inplanes, planes, kernel_size, stride, 
                 expand_rate=1.0, se_rate=0.25, 
                 drop_connect_rate=0.2):
        super(MBConv, self).__init__()

        expand_planes = int(inplanes * expand_rate)
        se_planes = max(1, int(inplanes * se_rate))

        self.expansion_conv = None        
        if expand_rate > 1.0:
            self.expansion_conv = nn.Sequential(
                nn.Conv2d(inplanes, expand_planes, 
                          kernel_size=1, stride=1, padding=0, bias=False),
                nn.BatchNorm2d(expand_planes, momentum=0.01, eps=1e-3),
                Swish()
            )
            inplanes = expand_planes

        self.depthwise_conv = nn.Sequential(
            nn.Conv2d(inplanes, expand_planes,
                      kernel_size=kernel_size, stride=stride, 
                      padding=kernel_size // 2, groups=expand_planes,
                      bias=False),
            nn.BatchNorm2d(expand_planes, momentum=0.01, eps=1e-3),
            Swish()
        )

        self.squeeze_excitation = SqueezeExcitation(expand_planes, se_planes)
        
        self.project_conv = nn.Sequential(
            nn.Conv2d(expand_planes, planes, 
                      kernel_size=1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(planes, momentum=0.01, eps=1e-3),
        )

        self.with_skip = stride == 1
        self.drop_connect_rate = torch.tensor(drop_connect_rate, requires_grad=False)
    
    def _drop_connect(self, x):        
        keep_prob = 1.0 - self.drop_connect_rate
        drop_mask = torch.rand(x.shape[0], 1, 1, 1) + keep_prob
        drop_mask = drop_mask.type_as(x)
        drop_mask.floor_()
        return drop_mask * x / keep_prob
        
    def forward(self, x):
        z = x
        if self.expansion_conv is not None:
            x = self.expansion_conv(x)

        x = self.depthwise_conv(x)
        x = self.squeeze_excitation(x)
        x = self.project_conv(x)
        
        # Add identity skip
        if x.shape == z.shape and self.with_skip:            
            if self.training and self.drop_connect_rate is not None:
                self._drop_connect(x)
            x += z
        return x

In [7]:
def init_weights(module):    
    if isinstance(module, nn.Conv2d):    
        nn.init.kaiming_normal_(module.weight, a=0, mode='fan_out')
    elif isinstance(module, nn.Linear):
        init_range = 1.0 / math.sqrt(module.weight.shape[1])
        nn.init.uniform_(module.weight, a=-init_range, b=init_range)
        
        
class EfficientNet(nn.Module):
        
    def _setup_repeats(self, num_repeats):
        return int(math.ceil(self.depth_coefficient * num_repeats))
    
    def _setup_channels(self, num_channels):
        num_channels *= self.width_coefficient
        new_num_channels = math.floor(num_channels / self.divisor + 0.5) * self.divisor
        new_num_channels = max(self.divisor, new_num_channels)
        if new_num_channels < 0.9 * num_channels:
            new_num_channels += self.divisor
        return new_num_channels

    def __init__(self, num_classes=10, 
                 width_coefficient=1.0,
                 depth_coefficient=1.0,
                 se_rate=0.25,
                 dropout_rate=0.2,
                 drop_connect_rate=0.2):
        super(EfficientNet, self).__init__()
        
        self.width_coefficient = width_coefficient
        self.depth_coefficient = depth_coefficient
        self.divisor = 8
                
        list_channels = [32, 16, 24, 40, 80, 112, 192, 320, 1280]
        list_channels = [self._setup_channels(c) for c in list_channels]
                
        list_num_repeats = [1, 2, 2, 3, 3, 4, 1]
        list_num_repeats = [self._setup_repeats(r) for r in list_num_repeats]        
        
        expand_rates = [1, 6, 6, 6, 6, 6, 6]
        strides = [1, 2, 2, 2, 1, 2, 1]
        kernel_sizes = [3, 3, 5, 3, 5, 5, 3]

        # Define stem:
        self.stem = nn.Sequential(
            nn.Conv2d(3, list_channels[0], kernel_size=3, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(list_channels[0], momentum=0.01, eps=1e-3),
            Swish()
        )
        
        # Define MBConv blocks
        blocks = []
        counter = 0
        num_blocks = sum(list_num_repeats)
        for idx in range(7):
            
            num_channels = list_channels[idx]
            next_num_channels = list_channels[idx + 1]
            num_repeats = list_num_repeats[idx]
            expand_rate = expand_rates[idx]
            kernel_size = kernel_sizes[idx]
            stride = strides[idx]
            drop_rate = drop_connect_rate * counter / num_blocks
            
            name = "MBConv{}_{}".format(expand_rate, counter)
            blocks.append((
                name,
                MBConv(num_channels, next_num_channels, 
                       kernel_size=kernel_size, stride=stride, expand_rate=expand_rate, 
                       se_rate=se_rate, drop_connect_rate=drop_rate)
            ))
            counter += 1
            for i in range(1, num_repeats):                
                name = "MBConv{}_{}".format(expand_rate, counter)
                drop_rate = drop_connect_rate * counter / num_blocks                
                blocks.append((
                    name,
                    MBConv(next_num_channels, next_num_channels, 
                           kernel_size=kernel_size, stride=1, expand_rate=expand_rate, 
                           se_rate=se_rate, drop_connect_rate=drop_rate)                                    
                ))
                counter += 1
        
        self.blocks = nn.Sequential(OrderedDict(blocks))
        
        # Define head
        self.head = nn.Sequential(
            nn.Conv2d(list_channels[-2], list_channels[-1], 
                      kernel_size=1, bias=False),
            nn.BatchNorm2d(list_channels[-1], momentum=0.01, eps=1e-3),
            Swish(),
            nn.AdaptiveAvgPool2d(1),
            Flatten(),
            nn.Dropout(p=dropout_rate),
            nn.Linear(list_channels[-1], num_classes)
        )

        self.apply(init_weights)
        
        self.sig = nn.Sigmoid()

    def forward(self, x):
        f = self.stem(x)
        f = self.blocks(f)
        y = self.head(f)
        y = self.sig(y)

        return y

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = EfficientNet(num_classes=1, 
                     width_coefficient=1.0, depth_coefficient=1.0, 
                     dropout_rate=0.2).to(device)

optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.BCELoss()

In [17]:
out_label = []
true_label = []

model_dir = '/content/MODEL_PTH_FILE'

def load_mode(model, model_dir):
  model_path = os.path.join(model_dir, 'model.pth')
  with open(model_path, 'rb') as f:
      model.load_state_dict(torch.load(f))  
  return model

# Provided model saving functions
def save_model(model, model_dir):
  print("Saving the model.")
  if not os.path.exists(model_dir):
    os.makedirs(model_dir)
  path = os.path.join(model_dir, 'model.pth')
  # save state dictionary
  torch.save(model.cpu().state_dict(), path)
    
# Provided train function
def train(model, train_loader, epochs, optimizer, criterion, device):
  """
  This is the training method that is called by the PyTorch training script. The parameters
  passed are as follows:
  model        - The PyTorch model that we wish to train.
  train_loader - The PyTorch DataLoader that should be used during training.
  epochs       - The total number of epochs to train for.
  optimizer    - The optimizer to use during training.
  criterion    - The loss function used for training. 
  device       - Where the model and data should be loaded (gpu or cpu).
  """
  
  for epoch in range(1, epochs + 1):
      model.train()
      total_loss = 0
      local__out_label = []
      local__out_target = []
      for batch_idx, (data, target) in enumerate(train_loader, 1):
          # prep data
          data, target = data.to(device), target.to(device)
          optimizer.zero_grad() # zero accumulated gradients
          # get output of SimpleNet
          output = model(data)
          # calculate loss and perform backprop
          
          loss = criterion(output, target.reshape(10, 1))

          loss.backward()
          optimizer.step()
  
          total_loss += loss.item()
          
          out_np = output.cpu().detach().numpy()
          local__out_label.append(np.squeeze(out_np.round()))
          out_target = target.cpu().detach().numpy()
          local__out_target.append(np.squeeze(out_target))

      out_label = np.concatenate( local__out_label, axis=0 )
      true_label = np.concatenate( local__out_target, axis=0)
      train_accuracy=accuracy_score(true_label, out_label) *100   

      # print loss stats
      print("Epoch: {}, Loss: {}".format(epoch, total_loss / len(train_loader) * 100), 'train accuracy: ', train_accuracy, '%')


In [34]:
train(model, train_loader, 100, optimizer, criterion, device)
# Save model data
save_model(model, model_dir)

Epoch: 1, Loss: 0.7647614330349437 train accuracy:  99.76666666666667 %
Saving the model.


In [11]:
# For loading the model, uncomment this cell
model = load_mode(model, model_dir)

In [37]:
correct = 0
total = 0
out_label = []
true_label = []


# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    model.eval().to(device)
    local__out_label = []
    local__out_target = []

    for batch_idx, (data, target) in enumerate(test_loader, 1):
        data, target = data.to(device), target.to(device)
        # calculate outputs by running images through the network
        outputs = model(data.to(device))
        
        out_np = outputs.cpu().detach().numpy()
        local__out_label.append(np.squeeze(out_np.round()))
        		    
        out_target = target.cpu().detach().numpy()
        local__out_target.append(np.squeeze(out_target))

out_label = np.concatenate(local__out_label, axis=0)
true_label = np.concatenate(local__out_target, axis=0 )

test_accuracy=accuracy_score(true_label, out_label)
print('Test accuracy: ', test_accuracy*100, '% ')

Test accuracy:  94.8051948051948 % 


In [36]:

activation = {}
inputs = {}

def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
        inputs[name] = input
    return hook



## This section is for exporting MBConv layers starting from 6_1
# Convolution for all MBConv*_* layers
def get_layer_matrices_conv2d(layer, dict_name):
  if dict_name == "MBConv1_0":
    print("No expansion layer for this version: Conv2d")
  else:
    layer.expansion_conv[0].register_forward_hook(get_activation('expansion_conv' + dict_name))
  layer.depthwise_conv[0].register_forward_hook(get_activation('depthwise_conv' + dict_name))
  layer.squeeze_excitation.reduce_expand[0].register_forward_hook(get_activation('squeeze_excitation1' + dict_name))
  layer.squeeze_excitation.reduce_expand[2].register_forward_hook(get_activation('squeeze_excitation2' + dict_name))
  layer.project_conv[0].register_forward_hook(get_activation('project_conv' + dict_name))

# BatchNorm for all MBConv*_* layers
def get_layer_matrices_BN(layer, dict_name):
  if dict_name == "MBConv1_0":
    print("No expansion layer for this version: BN")
  else:  
    layer.expansion_conv[1].register_forward_hook(get_activation('expansion_conv_BN' + dict_name))
  layer.depthwise_conv[1].register_forward_hook(get_activation('depthwise_conv_BN' + dict_name))
  layer.project_conv[1].register_forward_hook(get_activation('project_conv_BN' + dict_name))

# Activation for all MBConv*_* layers
def get_layer_matrices_activation(layer, dict_name):
  if dict_name == "MBConv1_0":
    print("No expansion layer for this version: Activation")
  else:  
    layer.expansion_conv[2].register_forward_hook(get_activation('expansion_conv_swish' + dict_name))
  layer.depthwise_conv[2].register_forward_hook(get_activation('depthwise_conv_swish' + dict_name))
  layer.squeeze_excitation.reduce_expand[1].register_forward_hook(get_activation('squeeze_excitation1_swish' + dict_name))
  layer.squeeze_excitation.reduce_expand[3].register_forward_hook(get_activation('squeeze_excitation2_sigmoid' + dict_name))

#Stem data
model.stem[0].register_forward_hook(get_activation('stem'))
model.stem[1].register_forward_hook(get_activation('stem_BN'))
model.stem[2].register_forward_hook(get_activation('stem_swish'))


# MBConv1_0 all the details
get_layer_matrices_conv2d(model.blocks.MBConv1_0, "MBConv1_0")
get_layer_matrices_BN(model.blocks.MBConv1_0, "MBConv1_0")
get_layer_matrices_activation(model.blocks.MBConv1_0, "MBConv1_0")

# MBConv6_1 all the details
get_layer_matrices_conv2d(model.blocks.MBConv6_1, "MBConv6_1")
get_layer_matrices_BN(model.blocks.MBConv6_1, "MBConv6_1")
get_layer_matrices_activation(model.blocks.MBConv6_1, "MBConv6_1")

# MBConv6_2 all the details
get_layer_matrices_conv2d(model.blocks.MBConv6_2, "MBConv6_2")
get_layer_matrices_BN(model.blocks.MBConv6_2, "MBConv6_2")
get_layer_matrices_activation(model.blocks.MBConv6_2, "MBConv6_2")

# MBConv6_3 all the details
get_layer_matrices_conv2d(model.blocks.MBConv6_3, "MBConv6_3")
get_layer_matrices_BN(model.blocks.MBConv6_3, "MBConv6_3")
get_layer_matrices_activation(model.blocks.MBConv6_3, "MBConv6_3")

# MBConv6_4 all the details
get_layer_matrices_conv2d(model.blocks.MBConv6_4, "MBConv6_4")
get_layer_matrices_BN(model.blocks.MBConv6_4, "MBConv6_4")
get_layer_matrices_activation(model.blocks.MBConv6_4, "MBConv6_4")

# MBConv6_5 all the details
get_layer_matrices_conv2d(model.blocks.MBConv6_5, "MBConv6_5")
get_layer_matrices_BN(model.blocks.MBConv6_5, "MBConv6_5")
get_layer_matrices_activation(model.blocks.MBConv6_5, "MBConv6_5")

# MBConv6_6 all the details
get_layer_matrices_conv2d(model.blocks.MBConv6_6, "MBConv6_6")
get_layer_matrices_BN(model.blocks.MBConv6_6, "MBConv6_6")
get_layer_matrices_activation(model.blocks.MBConv6_6, "MBConv6_6")

# MBConv6_7 all the details
get_layer_matrices_conv2d(model.blocks.MBConv6_7, "MBConv6_7")
get_layer_matrices_BN(model.blocks.MBConv6_7, "MBConv6_7")
get_layer_matrices_activation(model.blocks.MBConv6_7, "MBConv6_7")

# MBConv6_8 all the details
get_layer_matrices_conv2d(model.blocks.MBConv6_8, "MBConv6_8")
get_layer_matrices_BN(model.blocks.MBConv6_8, "MBConv6_8")
get_layer_matrices_activation(model.blocks.MBConv6_8, "MBConv6_8")

# MBConv6_9 all the details
get_layer_matrices_conv2d(model.blocks.MBConv6_9, "MBConv6_9")
get_layer_matrices_BN(model.blocks.MBConv6_9, "MBConv6_9")
get_layer_matrices_activation(model.blocks.MBConv6_9, "MBConv6_9")

# MBConv6_10 all the details
get_layer_matrices_conv2d(model.blocks.MBConv6_10, "MBConv6_10")
get_layer_matrices_BN(model.blocks.MBConv6_10, "MBConv6_10")
get_layer_matrices_activation(model.blocks.MBConv6_10, "MBConv6_10")

# MBConv6_11 all the details
get_layer_matrices_conv2d(model.blocks.MBConv6_11, "MBConv6_11")
get_layer_matrices_BN(model.blocks.MBConv6_11, "MBConv6_11")
get_layer_matrices_activation(model.blocks.MBConv6_11, "MBConv6_11")

# MBConv6_12 all the details
get_layer_matrices_conv2d(model.blocks.MBConv6_12, "MBConv6_12")
get_layer_matrices_BN(model.blocks.MBConv6_12, "MBConv6_12")
get_layer_matrices_activation(model.blocks.MBConv6_12, "MBConv6_12")

# MBConv6_13 all the details
get_layer_matrices_conv2d(model.blocks.MBConv6_13, "MBConv6_13")
get_layer_matrices_BN(model.blocks.MBConv6_13, "MBConv6_13")
get_layer_matrices_activation(model.blocks.MBConv6_13, "MBConv6_13")

# MBConv6_14 all the details
get_layer_matrices_conv2d(model.blocks.MBConv6_14, "MBConv6_14")
get_layer_matrices_BN(model.blocks.MBConv6_14, "MBConv6_14")
get_layer_matrices_activation(model.blocks.MBConv6_14, "MBConv6_14")

# MBConv6_15 all the details
get_layer_matrices_conv2d(model.blocks.MBConv6_15, "MBConv6_15")
get_layer_matrices_BN(model.blocks.MBConv6_15, "MBConv6_15")
get_layer_matrices_activation(model.blocks.MBConv6_15, "MBConv6_15")

#head data
model.head[0].register_forward_hook(get_activation('head'))
model.head[1].register_forward_hook(get_activation('head_BN'))
model.head[2].register_forward_hook(get_activation('head_swish'))
model.head[3].register_forward_hook(get_activation('head_pooling'))
model.head[4].register_forward_hook(get_activation('head_flatten'))
model.head[5].register_forward_hook(get_activation('head_dropOut'))
model.head[6].register_forward_hook(get_activation('head_linear'))

No expansion layer for this version: Conv2d
No expansion layer for this version: BN
No expansion layer for this version: Activation


In [38]:
## This cell for Conv2d parameters only fot a sublayer_name

%%capture cap --no-stderr
torch.set_printoptions(precision=8, sci_mode=False)

sublayer_name = "MBConv6_2"
layer_attr = getattr(model.blocks, sublayer_name)

def reshape_input(input, name):
  mat_input = input
  print(name + " has a shape of", mat_input[0].shape)
  mat_input_reshaped = torch.reshape(mat_input[0], (1, 1, 1, mat_input[0].numel()))
  print(name + " matrix is ", mat_input_reshaped)
  
# Weight shapes function  
def print_weight_shapes(layer, name):
  if sublayer_name == "MBConv1_0":
    print("No expansion wegihts to print")
  else:  
    print('Weight dimensions for conv2d operations')
    print(name, "\n", "Weight matrix shape = ", layer.expansion_conv[0].weight.shape)
    print(name, "\n", "Weight matrix shape = ", layer.depthwise_conv[0].weight.shape)
    print(name, "\n", "Weight matrix shape = ", layer.squeeze_excitation.reduce_expand[0].weight.shape)
    print(name, "\n", "Weight matrix shape = ", layer.squeeze_excitation.reduce_expand[2].weight.shape)
    print(name, "\n", "Weight matrix shape = ", layer.project_conv[0].weight.shape)

# Bias shapes function
def print_bias_shapes(layer, name):
  if sublayer_name == "MBConv1_0":
    print("No expansion weights to print")
  else:
    print('Bias dimensions for conv2d operations')
    if layer.expansion_conv[0].bias is not None:
      print(name, "\n", "Bias matrix shape = ", layer.expansion_conv[0].bias.shape)
    if layer.depthwise_conv[0].bias is not None:
      print(name, "\n", "Bias matrix shape = ", layer.depthwise_conv[0].bias.shape)
    if layer.squeeze_excitation.reduce_expand[0].bias is not None:
      print(name, "\n", "Bias matrix shape = ", layer.squeeze_excitation.reduce_expand[0].bias.shape)
    if layer.squeeze_excitation.reduce_expand[2].bias is not None:
      print(name, "\n", "Bias matrix shape = ", layer.squeeze_excitation.reduce_expand[2].bias.shape)
    if layer.project_conv[0].bias is not None:
      print(name, "\n", "Bias matrix shape = ", layer.project_conv[0].bias.shape)

# Weight tensor function
def permutate_weight_tensor(mat, name, par):
  if par == "DW":
    mat_weight = mat.weight.T.permute(3, 2, 1, 0)
    print(name, "\n", "has dimension of ", mat_weight.size())
    mat_weight_reshaped = mat_weight

  else:
    mat_weight = mat.weight.T.permute(1, 0, 3, 2)
    print(name, "\n", "has dimension of ", mat_weight.size())
    mat_weight_reshaped = torch.reshape(mat_weight, (1, 1, 1,mat_weight.numel()))
  print(mat_weight_reshaped)

# Bias tensor function 
def permutate_bias_tensor(mat, name):
  if mat.bias is not None:
    mat_bias = mat.bias.T
    print(name, "\n", "has dimension of ", mat_bias.size())
    mat_bias_reshaped = torch.reshape(mat_bias, (1, 1, 1,mat_bias.numel()))
    print(mat_bias_reshaped)


# Input Matrix to the current MBConv layer
print("Input Matrices to " + sublayer_name)
if sublayer_name == "MBConv1_0":
  print("No expansion input")
else:
  reshape_input(inputs['expansion_conv' + sublayer_name],       "Input for Expansion Convolution layer")
reshape_input(inputs['depthwise_conv' + sublayer_name],       "Input for Depthwise layer")
reshape_input(inputs['squeeze_excitation1' + sublayer_name],  "Input for Squeeze Excitation1 layer")
reshape_input(inputs['squeeze_excitation2' + sublayer_name],  "Input for Squeeze Excitation2 layer")
reshape_input(inputs['project_conv' + sublayer_name],         "Input for Project Conv layer")

# Weight Dimensions of the current MBConv layer
print('weight dimensions for conv2d operations')
print_weight_shapes(layer_attr, sublayer_name)

# Bias Dimensions of the current MBConv layer
print('Bias dimensions for conv2d operations')
print_bias_shapes(layer_attr, sublayer_name)

# Get weight and bias matrices of the current MBConv layer
print("Weight tensors")
if sublayer_name == "MBConv1_0":
  print("No expansion weight")
else:
  permutate_weight_tensor(layer_attr.expansion_conv[0], "weight for Expansion Convolution layer", "reg")
permutate_weight_tensor(layer_attr.depthwise_conv[0], "weight for Depthwise layer", "DW")
permutate_weight_tensor(layer_attr.squeeze_excitation.reduce_expand[0], "weight for Squeeze Excitation1 layer", "reg")
permutate_weight_tensor(layer_attr.squeeze_excitation.reduce_expand[2], "weight for Squeeze Excitation2 layer", "reg")
permutate_weight_tensor(layer_attr.project_conv[0], "Weight for Project Conv layer", "reg")

# Bias matrix pre-processing for current MBConv layer
print("Biasing tensors")
if sublayer_name == "MBConv1_0":
  print("No expansion bias")
else:
  permutate_bias_tensor(layer_attr.expansion_conv[0], "Bias for Expansion Convolution layer")
permutate_bias_tensor(layer_attr.depthwise_conv[0], "Bias for Depthwise layer")
permutate_bias_tensor(layer_attr.squeeze_excitation.reduce_expand[0], "Bias for Squeeze Excitation1 layer")
permutate_bias_tensor(layer_attr.squeeze_excitation.reduce_expand[2], "Bias for Squeeze Excitation2 layer")
permutate_bias_tensor(layer_attr.project_conv[0], "Bias for Project Conv layer")

# Output Matrix of current MBConv layer
print("Output Matrices to a whole MBConv layer", )
if sublayer_name == "MBConv1_0":
  print("No expansion output")
else:
  reshape_input(activation['expansion_conv' + sublayer_name],       "Output for Expansion Convolution layer")
reshape_input(activation['depthwise_conv' + sublayer_name],       "Output for Depthwise layer")
reshape_input(activation['squeeze_excitation1' + sublayer_name],  "Output for Squeeze Excitation1 layer")
reshape_input(activation['squeeze_excitation2' + sublayer_name],  "Output for Squeeze Excitation2 layer")
reshape_input(activation['project_conv' + sublayer_name],         "Output for Project Conv layer")

In [39]:
with open(sublayer_name + '_All_convolution_layers_in_this_layer'+'.txt', 'w') as out:
    out.write(cap.stdout)

In [40]:
## This cell for BN parameters only fot a sublayer_name

%%capture cap --no-stderr
torch.set_printoptions(precision=8, sci_mode=False)


# Print weight, bias, running mean, running variance
def WBRMRV(layer, name):
  print(name, "\n","Weight shape = ", layer.weight.shape)
  print("BatchNorm weight, bias, running mean and variance parameters: ", "\n" , layer.__dict__)

# Input Matrix to the current BN of convolutions in MBConv layer
print("Input Matrices to BN of " + sublayer_name)
if sublayer_name == "MBConv1_0":
  print("Input for BN of Expansion Convolution layer")
else:
  reshape_input(inputs['expansion_conv_BN' + sublayer_name],       "Input for BN of Expansion Convolution layer")
reshape_input(inputs['depthwise_conv_BN' + sublayer_name],       "Input for BN of Depthwise layer")
reshape_input(inputs['project_conv_BN' + sublayer_name],         "Input for BN of Project Conv layer")

# weights, bias, running mean and running variance
print("BatchNorm: weight, bias, running mean and running variance")
if sublayer_name == "MBConv1_0":
  print("Parameters for BN in Expansion Convolution layer")
else:
  WBRMRV(layer_attr.expansion_conv[1], "Parameters for BN in Expansion Convolution layer")
WBRMRV(layer_attr.depthwise_conv[1], "Parameters for BN in Depthwise layer")
WBRMRV(layer_attr.project_conv[1], "Parameters for BN in Project Conv layer")

# Output Matrix of current MBConv layer
print("Output Matrices of a BN layer of MBConv layer")
if sublayer_name == "MBConv1_0":
  print("No Output for BN of Expansion Convolution layer")
else:
  reshape_input(activation['expansion_conv_BN' + sublayer_name],       "Output for BN of Expansion Convolution layer")
reshape_input(activation['depthwise_conv_BN' + sublayer_name],       "Output for BN of Depthwise layer")
reshape_input(activation['project_conv_BN' + sublayer_name],         "Output for BN of Project Conv layer")

In [41]:
with open(sublayer_name + '_All_BatchNorm_in_this_layer'+'.txt', 'w') as out:
    out.write(cap.stdout)

In [42]:
## This cell for Swish activations only fot a sublayer_name

%%capture cap --no-stderr
torch.set_printoptions(precision=8, sci_mode=False)


print("Swish activation details ")

# Output Matrix of current MBConv layer
print("Input Matrices of an activation layer of MBConv layer")
if sublayer_name == "MBConv1_0":
  print("No Activation function input of Expansion Convolution layer")
else:
  reshape_input(inputs['expansion_conv_swish' + sublayer_name],       "Activation function input of Expansion Convolution layer")
reshape_input(inputs['depthwise_conv_swish' + sublayer_name],       "Activation function input of Depthwise layer")
reshape_input(inputs['squeeze_excitation1_swish' + sublayer_name],         "Activation function input for squeeze_excitation1_swish")
reshape_input(inputs['squeeze_excitation2_sigmoid' + sublayer_name],         "Activation function input for squeeze_excitation2_sigmoid")

# Output Matrix of current MBConv layer
print("Output Matrices of an activation layer of MBConv layer")
if sublayer_name == "MBConv1_0":
  print("No Activation function output of Expansion Convolution layer")
else:
  reshape_input(activation['expansion_conv_swish' + sublayer_name],       "Activation function output of Expansion Convolution layer")
reshape_input(activation['depthwise_conv_swish' + sublayer_name],       "Activation function output of Depthwise layer")
reshape_input(activation['squeeze_excitation1_swish' + sublayer_name],         "Activation function output for squeeze_excitation1_swish")
reshape_input(activation['squeeze_excitation2_sigmoid' + sublayer_name],         "Activation function output for squeeze_excitation2_sigmoid")

In [43]:
with open(sublayer_name + '_All_Activations_in_this_layer'+'.txt', 'w') as out:
    out.write(cap.stdout)

In [44]:
# This section is for stem layer
%%capture cap --no-stderr
torch.set_printoptions(precision=8, sci_mode=False)

####### model.blocks.stem[0].register_forward_hook(get_activation('stem'))
####### model.blocks.stem[1].register_forward_hook(get_activation('stem_BN'))
####### model.blocks.stem[2].register_forward_hook(get_activation('stem_swish'))

# Conv2d parameters: input, weight, output
reshape_input(inputs['stem'], "Input for stem_conv")
print("\n\nStem", "\n", "Weight matrix shape = ", model.stem[0].weight.shape)

mat_weight = model.stem[0].weight.T.permute(1, 0, 3, 2)
print("\n\nweight for stem layer has dimension of ", mat_weight.size(), "\n\n")
mat_weight_reshaped = torch.reshape(mat_weight, (1, 1, 1,mat_weight.numel()))
print(mat_weight_reshaped)

reshape_input(activation['stem'],       "Output for stem_conv layer")

# BN parameters: input, weight, bias, mean, variance, output
reshape_input(inputs['stem_BN'],       "Input for BN of stem layer")

WBRMRV(model.stem[1], "Parameters for BN in stem layer")

reshape_input(activation['stem_BN'],       "Output for BN of stem layer")

# Swish activation: Input, output
reshape_input(inputs['stem_swish'],         "Activation function input for stem_swish")

reshape_input(activation['stem_swish'],         "Activation function output for stem_swish")


In [45]:
with open("Stem" + '_All_details'+'.txt', 'w') as out:
    out.write(cap.stdout)

In [46]:
# This section is for head layer
%%capture cap --no-stderr
torch.set_printoptions(precision=8, sci_mode=False)

####### model.head[0].register_forward_hook(get_activation('head'))
####### model.head[1].register_forward_hook(get_activation('head'))
####### model.head[2].register_forward_hook(get_activation('head'))

# Conv2d parameters: input, weight, output
reshape_input(inputs['head'], "Input for head_conv")
print("\n\nhead", "\n", "Weight matrix shape = ", model.head[0].weight.shape)

mat_weight = model.head[0].weight.T.permute(1, 0, 3, 2)
print("\n\nweight for head layer has dimension of ", mat_weight.size(), "\n\n")
mat_weight_reshaped = torch.reshape(mat_weight, (1, 1, 1,mat_weight.numel()))
print(mat_weight_reshaped)

reshape_input(activation['head'],       "Output for head_conv layer")

# BN parameters: input, weight, bias, mean, variance, output
reshape_input(inputs['head_BN'],       "Input for BN of head layer")

WBRMRV(model.head[1], "Parameters for BN in head layer")

reshape_input(activation['head_BN'],       "Output for BN of head layer")

# Swish activation: Input, output
reshape_input(inputs['head_swish'],         "Activation function input for head_swish")

reshape_input(activation['head_swish'],         "Activation function output for head_swish")

# Adaptive pooling
reshape_input(inputs['head_pooling'],         "Pooling input for head_swish")

reshape_input(activation['head_pooling'],         "Pooling output for head_swish")

# Flatten
reshape_input(inputs['head_flatten'],         "Flatting input for head_swish")

reshape_input(activation['head_flatten'],         "Flatting output for head_swish")

# Dropout
reshape_input(inputs['head_dropOut'],         "Dropout input for head_swish")

reshape_input(activation['head_dropOut'],         "Dropout output for head_swish")

# Linear
reshape_input(inputs['head_linear'],         "Linear FC input for head_swish")

mat_weight = model.head[6].weight.T
print("\n\nweight for linear in head layer has dimension of ", mat_weight.size(), "\n\n")
mat_weight_reshaped = torch.reshape(mat_weight, (1, 1, 1,mat_weight.numel()))
print(mat_weight_reshaped)

mat_bias = model.head[6].bias.T
print("\n\bias for linear head layer has dimension of ", mat_bias.size(), "\n\n")
mat_bias_reshaped = torch.reshape(mat_bias, (1, 1, 1,mat_bias.numel()))
print(mat_bias_reshaped)

reshape_input(activation['head_linear'],         "Linear FC output for head_swish")

In [47]:
with open("head" + '_All_details'+'.txt', 'w') as out:
    out.write(cap.stdout)

In [ ]:
# Download the notebook
!pwd
!jupyter nbconvert --execute --to html '/content/drive/MyDrive/Colab Notebooks/Model_Trained_Extracted.ipynb'